In [190]:
from pickle import GLOBAL
import pinocchio as pin
import numpy as np
import time
import math
import sys
from numpy.linalg import norm, inv, pinv, det, matrix_rank
import pandas as pd

In [191]:
pinocchio_model_dir = '/home/unknown/Documents/Posdoc/ModelsCads/urdf' 
mesh_dir = pinocchio_model_dir + '/kinova_description/meshes/'
urdf_model_path =pinocchio_model_dir + '/j2s6s200_simple.urdf'
model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_model_path, mesh_dir)

In [192]:
frameIndex = model.getFrameId('j2s6s200_end_effector')
jointIndex = model.frames[frameIndex].parent
# Create data required by the algorithms
data     = model.createData()

In [ ]:
def mprint(M, name="ans",eps=1e-15):
    '''
    Matlab-style pretty matrix print.
    '''
    if isinstance(M, pin.SE3):
        M = M.homogeneous
    if len(M.shape) == 1:
        M = np.expand_dims(M, axis=0)
    ncol = M.shape[1]
    NC = 6
    print(name, " = ")
    print()

    Mmin = lambda M: M.min()
    Mmax = lambda M: M.max()
    Mm = Mmin(abs(M[np.nonzero(M)]))
    MM = Mmax(abs(M[np.nonzero(M)]))

    fmt = "% 10.4f" if Mm < 1e-5 or MM > 1e6 or MM / Mm > 1e3 else "% 1.5f"
    for i in range(math.floor((ncol - 1) / NC + 1)):
        cmin = i * 6
        cmax = (i + 1) * 6
        cmax = ncol if ncol < cmax else cmax
        print("Columns %s through %s" % (cmin, cmax - 1))
        print()
        for r in range(M.shape[0]):
            sys.stdout.write("  ")
            for c in range(cmin, cmax):
                if abs(M[r,c])>eps: sys.stdout.write(fmt % M[r,c]  + "   ")
                else: sys.stdout.write(" 0"+" "*9)
            print()
        print()

In [221]:
def run_robot(): 

    def df_dq(model,func,q,h=1e-9):
        """ Perform df/dq by num_diff. q is in the lie manifold.
        :params func: function to differentiate f : np.matrix -> np.matrix
        :params q: configuration value at which f is differentiated. type np.matrix
        :params h: eps
        
        :returns df/dq
        """
        #print("in")
        dq = pin.utils.zero(model.nv)
        f0 = func(q)
        AM_i = pin.utils.zero([6,3,3])
        for iq in range(model.nv):
            dq[iq] = h
            res2 = (func(pin.integrate(model,q,dq)) - f0)/h
            AM_i[iq,:,:] = np.array([ [res2[0],res2[3],res2[4]], [res2[3],res2[1],res2[5]], [res2[4],res2[5],res2[2]] ])
            dq[iq] = 0
        return AM_i
    
    def calc_jac3(q):
        """ Compute Jacobian """
        pin.forwardKinematics(model,data,q)
        pin.updateFramePlacements(model,data)
        pin.computeJointJacobians(model, data, q)
        J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED)
        Jbl = J[:3]
        R = Jbl@Jbl.T
        Rv = np.array([R[0][0],R[1][1], R[2][2], R[0][1], R[0][2], R[1][2]])                      
        return Rv.copy()

    # for runing the simulation
    t = 0
    dt = 1e-1

   
    md1 =15; md2 = 15;  md3 = 15
    mds = np.array([md1, md2, md3])
    Md = np.diag(mds) # Mass Matrix
    #Md_inv = np.linalg.pinv(Md)
    
    # for graphs
    E1_list = []
    manu_list = []
    q_list = [] #q
    pose_list = []
    manipaxes_list = []
    #manipaxesinv_list = []
    a = np.arange(50,180,5) #(47,180,5)
    b = np.arange(20,180,5) # (19,180,5)
    c = np.arange(65,295,5)
    contraint = pin.utils.zero(model.nv)

    ## for runing the simulation
    for k  in np.nditer(a):
        t += dt
        tic = time.time()
        q2  = np.deg2rad(k)
        for n in np.nditer(b):
            q3  = np.deg2rad(n)
            for p in np.nditer(c):
                q5  = np.deg2rad(p)
                q =np.array([0, q2, q3, np.pi, q5,0]) # Natural position as in the real case
                pin.forwardKinematics(model,data,q)
                pin.updateFramePlacements(model,data)
                OM_act = data.oMf[frameIndex] # This is the pose of the end-effector refered in the frame of the base

                pin.computeJointJacobians(model, data, q)
                J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED) # in the base frame
                Jbl = J[:3] # linear part of the Jacobian
                
                dJ_dqn = df_dq(model,lambda _q: calc_jac3(_q),q)
                pos = np.array([OM_act.translation[0],OM_act.translation[1],OM_act.translation[2]])
                x_vec = np.array([OM_act.translation[0], 0, 0])
                y_vec = np.array([0, OM_act.translation[1], 0])
                z_vec = np.array([0, 0, OM_act.translation[2]])                 
                #Jbl = check_matrix(Jbl)
                Al = Jbl@Jbl.T
                Al_inv = inv(Al)
                Alx = x_vec.T@(Jbl@Jbl.T)@x_vec
                Aly = y_vec.T@(Jbl@Jbl.T)@y_vec
                Alz = z_vec.T@(Jbl@Jbl.T)@z_vec

                Al_det = (det(Al))
                #Upsilon = 4*(tau_x/tau_q)*(Jbl@Jbl.T)
                if Al_det < 1e-15:  
                   Al_det = 0.0                
                if Al_det == 0.0:
                    contraint = np.array([math.nan,math.nan,math.nan,math.nan,math.nan,math.nan])
                else:
                    for i in range(len(q)):
                        #print("1")
                        contraint[i] = norm(dJ_dqn[i]@Al_inv,ord=2) # This should be less than 
                print(contraint)
                #print(contraint.dtype)
                #print(q.shape)
                #print(q.dtype)
                manip_linear = np.sqrt(Al_det)

                manip_axes = np.array([Alx,Aly,Alz])
                q_list.append(q)
                pose_list.append(pos)
                manu_list.append(manip_linear)
                E1_list.append(contraint)
                manipaxes_list.append(manip_axes)
                toc = time.time()
                ellapsed = toc - tic
                dt_sleep = max(0,dt - (ellapsed))
                time.sleep(dt_sleep)
                
    manu_list = np.array(manu_list)
    manipaxes_list = np.array(manipaxes_list)
    E1_list = np.array(E1_list)
    q_list = np.array(q_list)
    pose_list = np.array(pose_list)
    return  pose_list, q_list, manu_list, E1_list, manipaxes_list

In [222]:
pose, q, manip, constraint, manipaxes = run_robot()

[0.6905303  0.43608161 1.88406087 0.89613241 1.75876463 0.        ]
[0.4337875  0.72977654 2.17761107 0.96734781 1.75212311 0.        ]
[0.21954993 1.12156073 2.5469591  1.03486071 1.85691559 0.        ]
[0.09899943 1.61852869 3.01879398 1.09469622 2.06849981 0.        ]
[0.24797705 2.24413513 3.62680599 1.24342392 2.38079565 0.        ]
[0.46171189 3.03386294 4.41450251 1.49242728 2.80513206 0.        ]
[0.70147779 4.03744676 5.4397737  1.7715194  3.36789351 0.        ]
[0.96624508 5.32380867 6.78155739 2.07863474 4.10978395 0.        ]
[1.25503192 6.98840073 8.54903706 2.41431718 5.0888033  0.        ]


KeyboardInterrupt: 

In [220]:
print(min(constraint[:,0]))

2.6520917713475796


In [ ]:
pd.DataFrame(pose).to_csv('pose_workspace5.csv', index_label = "Index", header  = ['x','y','z'])    

In [ ]:
pd.DataFrame(manip).to_csv('manip_workspace5.csv', index_label = "Index", header  = ['manip'])    

In [ ]:
pd.DataFrame(q).to_csv('qs_workspace5.csv', index_label = "Index", header  = ['q1','q2','q3','q4','q5','q6'])    

In [ ]:
pd.DataFrame(E1).to_csv('E1_workspace5.csv', index_label = "Index", header  = ['Eq1','Eq2','Eq3','Eq4','Eq5','Eq6'])    

In [ ]:
pd.DataFrame(manipaxes).to_csv('manip_axes_workspace5.csv', index_label = "Index", header  = ['manipx','manipy','manipz'])    

In [ ]:
# tau_q = 0.375
# tau_x = 0.3
# tau_alpha = tau_x/tau_q
# kappa_min = 1/(4*tau_alpha)
# #print(kappa_min)
# a= 20.0
# x= 0.9 # for first tests, this is gonna be the positions of the end effector
# var = a*np.exp(-2.0*x)
# print(var)

In [ ]:
def df_dq(model,func,q,h=1e-9):
    """ Perform df/dq by num_diff. q is in the lie manifold.
    :params func: function to differentiate f : np.matrix -> np.matrix
    :params q: configuration value at which f is differentiated. type np.matrix
    :params h: eps
    
    :returns df/dq
    """
    dq = pin.utils.zero(model.nv)
    f0 = func(q)
    #print(f0)
    AM_i = pin.utils.zero([6,3,3])
    for iq in range(model.nv-1):
        dq[iq] = h
        res = (func(pin.integrate(model,q,dq)) - f0)/h 
        #print(res)
        AM_i[iq,:,:] = np.array([ [res[0],res[3],res[4]], [res[3],res[1],res[5]], [res[4],res[5],res[2]] ])
        dq[iq] = 0
    return AM_i.copy()

In [ ]:
# # Example from pinnochio
# def calc_vc(q,vq):
#     """ Compute COM velocity """
#     pin.centerOfMass(model,data,q,vq)
#     return data.vcom[0].copy()
# q = pin.randomConfiguration(model)
# vq = pin.utils.rand(model.nv)*2-1
# dvc_dqn = df_dq(model,lambda _q: calc_vc(_q,vq),q)
# mprint(dvc_dqn)

q = np.array([0, 2.9, 1.3, -2.07, 1.4,0]) # Natural position as in the real case
#q = np.array([np.pi, np.pi, np.pi, np.pi, np.pi,np.pi])
#q =  pin.randomConfiguration(model)
#qdot = pin.utils.zero(model.nv) # this is the angular velocity
#pin.computeJointJacobians(model, data, q)
#pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED)
#data.J[0]
#JJ=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED)
#Jbl = JJ[:3]
#R = Jbl@Jbl.T
#Rv = np.array([R[0][0],R[1][1], R[2][2], R[0][1], R[0][2], R[1][2]])
#mprint(Rv)dk_dq
pin.computeJointJacobians(model, data, q)
J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED) # in the base frame
Jbl = J[:3] # linear part of the Jacobian
a = 20
b =0.5
tau_q = 0.375
tau_x = 0.3
tau_alpha = tau_x/tau_q
kappa_min = 1/(4*tau_alpha)
def calc_jac3(q):
     """ Compute Jacobian """
     pin.forwardKinematics(model,data,q)
     pin.updateFramePlacements(model,data)
     pin.computeJointJacobians(model, data, q)
     J=pin.getFrameJacobian(model,data,frameIndex,pin.LOCAL_WORLD_ALIGNED)
     Jbl = J[:3]
     R = Jbl@Jbl.T
     Rv = np.array([R[0][0],R[1][1], R[2][2], R[0][1], R[0][2], R[1][2]])      
     #print(Rv)                
     return Rv.copy()

def calc_kappa(q):
     """ Compute Jacobian """
     pin.forwardKinematics(model,data,q)
     pin.updateFramePlacements(model,data)
     pin.computeJointJacobians(model, data, q)
     OM_act = data.oMf[frameIndex]
     posn = norm(OM_act.translation)
     kappas = a*np.exp(-b*posn)
     #print(kappas)
     Kappa = np.array([kappas,kappas,kappas,0,0,0])
     #print(Kappa)             
     return Kappa

dJ_dq = df_dq(model,lambda _q: calc_jac3(_q),q)
#dk_dq = df_dq(model,lambda _q: calc_kappa(_q),q)
Al_inv = inv(Jbl@Jbl.T)
EM_2 = pin.utils.zero(model.nv)
#EM_3 = pin.utils.zero([6,3,3])

for i in range(len(q)):
     EM_2[i] = norm(dJ_dq[i]@Al_inv,ord=2) # This should be less than 
     #EM_3[i,:,:] = dk_dq[i]*kappa_min # This should be less than

#res = calc_kappaM(q)

#E1 = pin.utils.zero(model.nv)
#for ic in range(len(q)):
#     E1[ic] = norm(dJ_dqn[ic],ord=2) # This should be less than 
     #print(E1)
#mprint(R,"R")
#mprint(Rv,"Rv")
#print(E1)
#print(res)
# a = 20
# b =4
# pin.forwardKinematics(model,data,q)
# pin.updateFramePlacements(model,data)
# pin.computeJointJacobians(model, data, q)
# OM_act = data.oMf[frameIndex]
# posn = norm(OM_act.translation)
# kappa = a*np.exp(-b*posn)
#print(dJ_dq)
#print(dJ_dq[5])
print(EM_2)

In [ ]:
#E1 = pin.utils.zero(model.nv)
#for i in range(len(q)):
E1 = np.array([math.nan,math.nan,math.nan,math.nan,math.nan,math.nan])
print(E1)    